# Text Source PubMed
Sales words

In [6]:
!python -m pip install openai
!python -m pip install pandas
!python -m pip install scipy
!python -m pip install requests
!python -m pip install numexpr

In [7]:
import sys
sys.path.append('/Users/andersohrn/opt/anaconda3/lib/python3.8/site-packages')
print (sys.path)

['/Users/andersohrn/Development/das_wort', '/Users/andersohrn/opt/anaconda3/lib/python38.zip', '/Users/andersohrn/opt/anaconda3/lib/python3.8', '/Users/andersohrn/opt/anaconda3/lib/python3.8/lib-dynload', '', '/Users/andersohrn/river_chatgpt/river_gpt/lib/python3.8/site-packages', '/Users/andersohrn/opt/anaconda3/lib/python3.8/site-packages', '/Users/andersohrn/opt/anaconda3/lib/python3.8/site-packages']


In [22]:
from typing import Optional, List
from datetime import date

import requests
import openai
import pandas as pd
from xml.etree import ElementTree as ET

In [70]:
PUBMED_APIs = {
    'standard' : {
        'base_url': 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/',
        'apis': {
            'search' : 'esearch.fcgi',
            'fetch' : 'efetch.fcgi'
        }
    },
    'open_access': {
        'base_url': 'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/'
    }
}

In [71]:
def create_basic_search_term(author: Optional[str]=None, 
                             title: Optional[str]=None, 
                             abstract: Optional[str]=None, 
                             published_after_date: Optional[date]=None):
    term = []
    if not author is None:
        term.append('{}[Author]'.format(author))
    if not title is None:
        term.append('{}[Title]'.format(title))
    if not abstract is None:
        term.append('{}[Title/Abstract]'.format(abstract))
    if not published_after_date is None:
        term.append('{}[Date]'.format(FOOBAR(published_after_date)))
        
    if len(term) == 0:
        raise ValueError('No input values given')
        
    return '+AND+'.join(term)

search_term = create_basic_search_term(title='HER2-positive', author='Akihito Kawazoe')

In [72]:
def do_search_via_api(search_term: str):
    url = '{}{}?db=pubmed'.format(
        PUBMED_APIs['standard']['base_url'], 
        PUBMED_APIs['standard']['apis']['search']
    )
    url_full = '{}&term={}'.format(url, search_term)
    print('URL to call: {}'.format(url_full))
    
    r = requests.get(url=url_full)
    if r.status_code != 200:
        print ('Status code {} received!'.format(status_code))
        print ('Error message: {}'.format(r.content))
        
    return r.content

pubmed_object = do_search_via_api(search_term)

URL to call: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Akihito Kawazoe[Author]+AND+HER2-positive[Title]


In [73]:
root = ET.fromstring(pubmed_object)
pubmed_ids = [x.text for x in root.findall('.//Id')]
print(pubmed_ids)

['34912120']


In [74]:
def get_abstracts_via_api(ids: List[str]):
    url = '{}{}?db=pubmed'.format(
        PUBMED_APIs['standard']['base_url'], 
        PUBMED_APIs['standard']['apis']['fetch']
    )
    url_full = '{}&rettype=Abstract&id={}'.format(url, ','.join(ids))
    print('URL to call: {}'.format(url_full))
    
    r = requests.get(url=url_full)
    if r.status_code != 200:
        print ('Status code {} received!'.format(status_code))
        print ('Error message: {}'.format(r.content))
        
    return r.content

pubmed_abstract_object = get_abstracts_via_api(ids=['34912120'])

URL to call: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&rettype=Abstract&id=34912120


In [92]:
root = ET.fromstring(pubmed_abstract_object)
root_dict = element_to_dict(root)

In [95]:

# Function to extract text recursively from an element
def extract_text(element):
    text = element.text or ''
    for child in element:
        text += extract_text(child)
    text += element.tail or ''
    return text

# Find the AbstractText element and extract its text content
abstract_texts = []
abstract_text_elements = root.findall('.//AbstractText')
for element in abstract_text_elements:
    abstract_text extract_text(element)

# Print the extracted text
print(abstract_text)

Human epidermal growth factor receptor 2 (HER2, also known as ERBB2) amplification or overexpression occurs in approximately 20% of advanced gastric or gastro-oesophageal junction adenocarcinomas1-3. More than a decade ago, combination therapy with the anti-HER2 antibody trastuzumab and chemotherapy became the standard first-line treatment for patients with these types of tumours4. Although adding the anti-programmed death 1 (PD-1) antibody pembrolizumab to chemotherapy does not significantly improve efficacy in advanced HER2-negative gastric cancer5, there are preclinical6-19 and clinical20,21 rationales for adding pembrolizumab in HER2-positive disease. Here we describe results of the protocol-specified first interim analysis of the randomized, double-blind, placebo-controlled phase III KEYNOTE-811 study of pembrolizumab plus trastuzumab and chemotherapy for unresectable or metastatic, HER2-positive gastric or gastro-oesophageal junction adenocarcinoma22 ( https://clinicaltrials.gov 

In [52]:
print(get_element(root_dict, ('PubmedArticleSet','@children','PubmedArticle','@children','MedlineCitation','@children','Article','@children','Abstract','@children','AbstractText','@text')))

[[[[['Human epidermal growth factor receptor 2 (HER2,\xa0also known as ERBB2) amplification or overexpression occurs in approximately 20% of advanced gastric or gastro-oesophageal junction adenocarcinomas']]]]]


[[[[['Human epidermal growth factor receptor 2 (HER2,\xa0also known as ERBB2) amplification or overexpression occurs in approximately 20% of advanced gastric or gastro-oesophageal junction adenocarcinomas']]]]]
